In [0]:
!pip install lightgbm

In [0]:
#@title
#coding:utf-8
import os

# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
os.chdir('/content/drive/My Drive/Colab/人工智慧期末')
!ls

my_model1.h5  test_fake_news.csv   人工智慧期末.ipynb
my_model.h5   train_fake_news.csv


In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#1: unreliable
#0: reliable
train=pd.read_csv('train_fake_news.csv')
test=pd.read_csv('test_fake_news.csv')
test.info()
test['label']='t'
train.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5200 entries, 0 to 5199
Data columns (total 4 columns):
id        5200 non-null int64
title     5078 non-null object
author    4697 non-null object
text      5193 non-null object
dtypes: int64(1), object(3)
memory usage: 162.6+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
id        20800 non-null int64
title     20242 non-null object
author    18843 non-null object
text      20761 non-null object
label     20800 non-null int64
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [0]:
train

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",0
6,6,Life: Life Of Luxury: Elton John’s 6 Favorite ...,NaN,Ever wonder how Britain’s most iconic pop pian...,1
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi...",0
8,8,Excerpts From a Draft Script for Donald Trump’...,NaN,Donald J. Trump is scheduled to make a highly ...,0
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...,0


In [0]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import HashingVectorizer
import re
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')




def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
                           text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) \
                   + ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

vect = HashingVectorizer(decode_error='ignore',
                         n_features=2**21,
                         preprocessor=None,
                         tokenizer=tokenizer)
stop = stopwords.words('english')




#data prep
test=test.fillna(' ')
train=train.fillna(' ')
test['total']=test['title']+' '+test['author']+test['text']
train['total']=train['title']+' '+train['author']+train['text']

#tfidf
#transformer = TfidfTransformer(smooth_idf=False)
#count_vectorizer = CountVectorizer(ngram_range=(1, 2))
#counts = count_vectorizer.fit_transform(train['total'].values)
#tfidf = transformer.fit_transform(counts)
tfidf = vect.transform(train['total'].values)

targets = train['label'].values
test_counts = vect.transform(test['total'].values)
#test_counts = transformer.fit_transform(test_counts)

#split in samples
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(tfidf, targets, random_state=0)

#------------------------------------------------------------------------------------------------------------
from sklearn.ensemble import (RandomForestClassifier, ExtraTreesClassifier,
                              AdaBoostClassifier)

Extr = ExtraTreesClassifier(n_estimators=5,n_jobs=4)
clf = Extr.fit(X_train, y_train)
print('Accuracy of ExtrTrees classifier on training set: {:.2f}'
     .format(Extr.score(X_train, y_train)))
print('Accuracy of Extratrees classifier on test set: {:.2f}'
     .format(Extr.score(X_test, y_test)))




[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Accuracy of ExtrTrees classifier on training set: 1.00
Accuracy of Extratrees classifier on test set: 0.80


In [0]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(C=1e5)
clf = logreg.fit(X_train, y_train)
print('Accuracy of Lasso classifier on training set: {:.2f}'
     .format(logreg.score(X_train, y_train)))
print('Accuracy of Lasso classifier on test set: {:.2f}'
     .format(logreg.score(X_test, y_test)))

predictions = clf.predict(X_test)

print(classification_report(y_test, predictions))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Accuracy of Lasso classifier on training set: 1.00
Accuracy of Lasso classifier on test set: 0.97
              precision    recall  f1-score   support

           0       0.98      0.96      0.97      2564
           1       0.96      0.98      0.97      2636

   micro avg       0.97      0.97      0.97      5200
   macro avg       0.97      0.97      0.97      5200
weighted avg       0.97      0.97      0.97      5200



In [0]:
from xgboost.sklearn import XGBClassifier

xgb = XGBClassifier()

clf = xgb.fit(X_train, y_train)
print('Accuracy of XGBClassifier on training set: {:.2f}'
     .format(xgb.score(X_train, y_train)))
print('Accuracy of XGBClassifier on test set: {:.2f}'
     .format(xgb.score(X_test, y_test)))

predictions = clf.predict(X_test)

print(classification_report(y_test, predictions))

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Accuracy of XGBClassifier on training set: 0.98


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Accuracy of XGBClassifier on test set: 0.97
             precision    recall  f1-score   support

          0       0.98      0.95      0.96      2564
          1       0.96      0.98      0.97      2636

avg / total       0.97      0.97      0.97      5200



/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [0]:
from sklearn.svm import SVC
svc = SVC(C=3, probability=True)
clf = svc .fit(X_train, y_train)
print('Accuracy of svc  on training set: {:.2f}'
     .format(svc.score(X_train, y_train)))
print('Accuracy of svc  on test set: {:.2f}'
     .format(svc.score(X_test, y_test)))

predictions = clf.predict(X_test)

print(classification_report(y_test, predictions))

In [0]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.ensemble import BaggingClassifier      # Bagging 每個模型都不錯 做 Voting/Avg 找出最好的 或是給權重  ex:RF
from sklearn.ensemble import AdaBoostClassifier     # Boosting 給數個弱學習器 適合不夠強的模型

bag = BaggingClassifier(base_estimator=clf, n_estimators=100,max_samples=0.5, max_features=0.5)
clf1 = bag.fit(X_train, y_train)
print('Accuracy of BaggingClassifier on training set: {:.2f}'
     .format(bag.score(X_train,  y_train)))
print('Accuracy of BaggingClassifier on test set: {:.2f}'
     .format(bag.score(X_test, y_test)))

Accuracy of BaggingClassifier on training set: 1.00
Accuracy of BaggingClassifier on test set: 0.97


In [0]:
from lightgbm import LGBMClassifier
lgbm = LGBMClassifier(boosting_type= 'gbdt', objective = 'binary')
clf = lgbm.fit(X_train, y_train)
print('Accuracy of LGBMClassifier on training set: {:.2f}'
     .format(lgbm.score(X_train, y_train)))
print('Accuracy of LGBMClassifier on test set: {:.2f}'
     .format(lgbm.score(X_test, y_test)))

predictions = clf.predict(X_test)

print(classification_report(y_test, predictions))

Accuracy of LGBMClassifier on training set: 1.00
Accuracy of LGBMClassifier on test set: 0.98
              precision    recall  f1-score   support

           0       0.98      0.98      0.98      2564
           1       0.98      0.98      0.98      2636

   micro avg       0.98      0.98      0.98      5200
   macro avg       0.98      0.98      0.98      5200
weighted avg       0.98      0.98      0.98      5200



In [0]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.ensemble import BaggingClassifier      # Bagging 每個模型都不錯 做 Voting/Avg 找出最好的 或是給權重  ex:RF
from sklearn.ensemble import AdaBoostClassifier     # Boosting 給數個弱學習器 適合不夠強的模型

#eclf1 = VotingClassifier(estimators=[('lr', lgbm), ('rf', rf), ('nb',xgb2), ('gb', gb), ('svc',svc), ('nn',nn)], voting='soft')
eclf1 = VotingClassifier(estimators=[('logreg',logreg),('lgbm',lgbm)], voting='soft')
#eclf1 = eclf1.fit(x_train, y_train)
#score = eclf1.score(x_val, y_val)
#cross_val_score(estimator=clf,X=x_val,y=y_val,cv=3)

#bag = AdaBoostClassifier(base_estimator=eclf1, n_estimators=50)
#bag = BaggingClassifier(base_estimator=eclf1, n_estimators=2,max_samples=0.5, max_features=0.5)

clf = eclf1.fit(X_train, y_train)
print('Accuracy of VotingClassifier on training set: {:.2f}'
     .format(eclf1.score(X_train, y_train)))
print('Accuracy of VotingClassifier on test set: {:.2f}'
     .format(eclf1.score(X_test, y_test)))


predictions = eclf1.predict(X_test)

print(classification_report(y_test, predictions))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Accuracy of VotingClassifier on training set: 1.00
Accuracy of VotingClassifier on test set: 0.98
              precision    recall  f1-score   support

           0       0.99      0.98      0.98      2564
           1       0.98      0.99      0.98      2636

   micro avg       0.98      0.98      0.98      5200
   macro avg       0.98      0.98      0.98      5200
weighted avg       0.98      0.98      0.98      5200



In [0]:
# It will oom

bag = AdaBoostClassifier(base_estimator=eclf1, n_estimators=50)

clf2 = bag.fit(X_train, y_train)
print('Accuracy of Majarity_Voting_AdaBoost on training set: {:.2f}'
     .format(bag.score(X_train, y_train)))
print('Accuracy of Majarity_Voting_AdaBoost on test set: {:.2f}'
     .format(bag.score(X_test, y_test)))


predictions = clf2.predict(X_test)

print(classification_report(y_test, predictions))

Accuracy of Majarity_Voting_AdaBoost on training set: 1.00
Accuracy of Majarity_Voting_AdaBoost on test set: 0.98
             precision    recall  f1-score   support

          0       0.99      0.98      0.98      2564
          1       0.98      0.99      0.98      2636

avg / total       0.98      0.98      0.98      5200



In [0]:
from sklearn.ensemble import BaggingClassifier 


bag2 = BaggingClassifier(base_estimator=eclf1, n_estimators=5)

clf3 = bag2.fit(X_train, y_train)
print('Accuracy of Majarity_Voting_AdaBoost on training set: {:.2f}'
     .format(bag2.score(X_train, y_train)))
print('Accuracy of Majarity_Voting_AdaBoost on test set: {:.2f}'
     .format(bag2.score(X_test, y_test)))


predictions = clf3.predict(X_test)

print(classification_report(y_test, predictions))

Accuracy of Majarity_Voting_AdaBoost on training set: 1.00
Accuracy of Majarity_Voting_AdaBoost on test set: 0.98
             precision    recall  f1-score   support

          0       0.99      0.98      0.98      2564
          1       0.98      0.99      0.98      2636

avg / total       0.98      0.98      0.98      5200



In [0]:
from sklearn.model_selection import GridSearchCV ,  RandomizedSearchCV
from lightgbm import LGBMClassifier

params = {
                 'n_estimators':[5,15,25,30,50]
             # 'max_depth': [100,35,25],
              #'learning_rate': [0.01,0.15,0.1],
              #'feature_fraction': [0.6, 0.95],
              #'bagging_fraction': [0.6, 0.7],
              #'bagging_freq': [2, 8],
              #'lambda_l1': [0,0.4],
              #'lambda_l2': [0, 10,40],
              #'cat_smooth': [1, 35]
          
}

bag2 = BaggingClassifier(base_estimator=eclf1, n_estimators=5)




grid = GridSearchCV(estimator=bag2, param_grid=params, cv=5,verbose=1,n_jobs=-1)
grid1 = grid.fit(X_train, y_train)


print('Best parameter set: %s ' % grid1.best_params_)


print('Accuracy of LGBMClassifier on training set: {:.2f}'
     .format(grid.score(X_train, y_train)))
print('Accuracy of LGBMClassifier on test set: {:.2f}'
     .format(grid.score(X_test, y_test)))

predictions = grid1.predict(X_test)

print(classification_report(y_test, predictions))


Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
exception calling callback for <Future at 0x7fa4305abef0 state=finished raised TerminatedWorkerError>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/externals/loky/_base.py", line 625, in _invoke_callbacks
    callback(self)
  File "/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/parallel.py", line 309, in __call__
    self.parallel.dispatch_next()
  File "/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/parallel.py", line 731, in dispatch_next
    if not self.dispatch_one_batch(self._original_iterator):
  File "/usr/local/lib/python

TerminatedWorkerError: ignored

In [0]:
from sklearn.model_selection import GridSearchCV ,  RandomizedSearchCV
from lightgbm import LGBMClassifier

params = {
                 'n_estimators':[800,1600,1200]
             # 'max_depth': [100,35,25],
              #'learning_rate': [0.01,0.15,0.1],
              #'feature_fraction': [0.6, 0.95],
              #'bagging_fraction': [0.6, 0.7],
              #'bagging_freq': [2, 8],
              #'lambda_l1': [0,0.4],
              #'lambda_l2': [0, 10,40],
              #'cat_smooth': [1, 35]
          
}

bag2 = BaggingClassifier(base_estimator=eclf1, n_estimators=5)




grid = GridSearchCV(estimator=lgbm, param_grid=params, cv=2,verbose=1,n_jobs=-1)
grid1 = grid.fit(X_train, y_train)


print('Best parameter set: %s ' % grid1.best_params_)


print('Accuracy of LGBMClassifier on training set: {:.2f}'
     .format(grid.score(X_train, y_train)))
print('Accuracy of LGBMClassifier on test set: {:.2f}'
     .format(grid.score(X_test, y_test)))

predictions = grid1.predict(X_test)

print(classification_report(y_test, predictions))



Fitting 2 folds for each of 3 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed: 22.1min finished


Best parameter set: {'n_estimators': 800} 
Accuracy of LGBMClassifier on training set: 1.00
Accuracy of LGBMClassifier on test set: 0.98
              precision    recall  f1-score   support

           0       0.99      0.98      0.98      2564
           1       0.98      0.99      0.99      2636

   micro avg       0.98      0.98      0.98      5200
   macro avg       0.98      0.98      0.98      5200
weighted avg       0.98      0.98      0.98      5200



In [0]:
os.chdir('/content/drive/My Drive/Colab/人工智慧期末')
!ls

test_fake_news.csv  train_fake_news.csv  人工智慧期末.ipynb


In [0]:
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

import pandas as pd
import os
import nltk
nltk.download('stopwords')

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#1: unreliable
#0: reliable
train=pd.read_csv('train_fake_news.csv')
test=pd.read_csv('test_fake_news.csv')
test['label']='t'
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import HashingVectorizer
import re
from nltk.corpus import stopwords
from sklearn.metrics import classification_report


#data prep
test=test.fillna(' ')
train=train.fillna(' ')
test['total']=test['title']+' '+test['author']+test['text']
train['total']=train['title']+' '+train['author']+train['text']
targets = train['label'].values
df5= train['total'].astype(str)
df5 = df5.values.tolist()
df5 = ["".join(map(str, lst)) for lst in df5]  # 變成 一個一個的 ROW
df6 = df5
type(df6[0])
import re #匯入Regular Expression模組
stop = stopwords.words('english')
from nltk.corpus import stopwords

result = list()
for line in df6:    

    line = re.sub('<[^>]*>', '', line)
    line1 = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
                          line.lower())
    
    line = re.sub('[\W]+', ' ', line.lower()) \
                   + ' '.join(line1).replace('-', '')
    #line = re.sub(r'(?::|;|=)(?:-)?(?:\)|\(|D|P)',' ', line) 
        
    pattern = re.compile(r'\b(' + r'|'.join(stop) + r')\b\s*')    # 去除停用字但可以分成一句一句
    line = pattern.sub('', line)
    
    
#    line = re.sub(r'[^\u4e00-\u9fa5p]',' ', line) 
#    line = re.sub(u"\\(.*?\\)|\\{.*?}|\\[.*?]", " ", line)   # 去除() {} [] 中的值
#    line = re.sub(r'\b(\w+)( \1\b)+', r'\1', line)           # 去除 重複的值
#    line = re.sub(r'[^\u4e00-\u9fa5p]',' ', line)            # 去除非中文的值
    
    result.append(line)  
token = Tokenizer(num_words=200000)
token.fit_on_texts(result)
x_train_seq = token.texts_to_sequences(result)
x_train = sequence.pad_sequences(x_train_seq, maxlen=1000,padding='post')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
'''from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation,Flatten
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional, GRU
from keras import backend as K
from keras.layers import Layer
from keras import initializers, regularizers, constraints
from keras.layers.normalization import BatchNormalization
import keras
from keras.models import Model 
from keras.layers import Dense, Input
from keras.layers.convolutional import Conv2D, Conv3D
from keras.layers import Add,Flatten,Concatenate, GlobalAveragePooling1D
from keras.layers.recurrent import LSTM

S_inputs = Input(shape=(None, ))  # 輸入層
embeddings1 = Embedding(output_dim=512,input_dim=200000,input_length=1000)(S_inputs)   

x = Conv1D(512, 1, strides=1, padding='valid', activation='relu')(embeddings1) 
x = Conv1D(filters=128, kernel_size=3, strides=1,padding='valid', activation='relu')(x)
x = Conv1D(512, 1, strides=1, padding='valid', activation='relu')(x) 
x = Conv1D(1, 1,strides=1, padding='valid', activation='sigmoid')(x) 

x = GlobalAveragePooling1D()(x)

model = Model(inputs=S_inputs, outputs=x)
model.summary()'''


from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation,Flatten
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional, GRU
from keras import backend as K
from keras.layers import Layer
from keras import initializers, regularizers, constraints
from keras.layers.normalization import BatchNormalization
import keras
 




model = Sequential()
model.add(Embedding(output_dim=32,
                    input_dim=200000, 
                    input_length=1000))

model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))




#model.add(keras.layers.core.Masking(mask_value=0., input_shape=(max_lenth, max_features)))
model.add(Bidirectional(GRU(units=16,activation='selu',kernel_initializer='orthogonal', recurrent_initializer='orthogonal',
              bias_initializer='zeros', kernel_regularizer=regularizers.l2(0.01), recurrent_regularizer=regularizers.l2(0.01),
              bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, recurrent_constraint=None,
              bias_constraint=None, dropout=0.5, recurrent_dropout=0.0, implementation=1, return_sequences=True,#多层时需设置为true
              return_state=False, go_backwards=False, stateful=False, unroll=False),merge_mode='concat'))   #input_shape=(max_lenth, max_features),
model.add(Bidirectional(GRU(units=16,activation='selu',kernel_initializer='orthogonal', recurrent_initializer='orthogonal',
              bias_initializer='zeros', kernel_regularizer=regularizers.l2(0.01), recurrent_regularizer=regularizers.l2(0.01),
              bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, recurrent_constraint=None,
              bias_constraint=None, dropout=0.5, recurrent_dropout=0.0, implementation=1, return_sequences=True,
              return_state=False, go_backwards=False, stateful=False, unroll=False),merge_mode='concat'))   #input_shape=(max_lenth, max_features),
model.add(Dropout(0.5))
model.add(AttentionWithContext())
model.add(Dense(1))
model.add(BatchNormalization())
model.add(keras.layers.core.Activation('sigmoid'))

In [0]:
K.clear_session()

In [0]:
#opt = keras.optimizers.adam(lr=0.001, decay=1e-6)

model.compile(loss='binary_crossentropy',  #使用compile對訊連模型進行設定,損失函數,在深度學習使用cross_entropy交叉熵
              optimizer= 'adam',  #設定訓練時的最優化方法,用adam,可以讓訓練更快收斂,提高準確率
              metrics=['accuracy']) #設定

model.fit(x_train, targets, batch_size=256, epochs=10, verbose=1,validation_split=0.1)

model.save('my_model.h5')

Train on 18720 samples, validate on 2080 samples
Epoch 1/10
18720/18720 [==============================] - 417s 22ms/step - loss: 0.9835 - acc: 0.9379 - val_loss: 0.6786 - val_acc: 0.9596
Epoch 2/10
18720/18720 [==============================] - 406s 22ms/step - loss: 0.5667 - acc: 0.9795 - val_loss: 0.4952 - val_acc: 0.9490
Epoch 3/10
18720/18720 [==============================] - 405s 22ms/step - loss: 0.3971 - acc: 0.9889 - val_loss: 0.4742 - val_acc: 0.9466
Epoch 4/10
18720/18720 [==============================] - 403s 22ms/step - loss: 0.3172 - acc: 0.9923 - val_loss: 0.4956 - val_acc: 0.9563
Epoch 5/10
18720/18720 [==============================] - 401s 21ms/step - loss: 0.2772 - acc: 0.9939 - val_loss: 0.5364 - val_acc: 0.9534
Epoch 6/10
18720/18720 [==============================] - 397s 21ms/step - loss: 0.2516 - acc: 0.9951 - val_loss: 0.5595 - val_acc: 0.9611
Epoch 7/10
18720/18720 [==============================] - 397s 21ms/step - loss: 0.2329 - acc: 0.9967 - val_loss: 0.6

In [0]:
model.save('my_model1.h5')

In [0]:
from keras import backend as K
from keras.layers import Layer
from keras import initializers, regularizers, constraints
 
def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)
 
 
class AttentionWithContext(Layer):
    """
    Attention operation, with a context/query vector, for temporal data.
    Supports Masking.
    Follows the work of Yang et al. [https://www.cs.cmu.edu/~diyiy/docs/naacl16.pdf]
    "Hierarchical Attention Networks for Document Classification"
    by using a context vector to assist the attention
    # Input shape
        3D tensor with shape: `(samples, steps, features)`.
    # Output shape
        2D tensor with shape: `(samples, features)`.
    How to use:
    Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
    The dimensions are inferred based on the output shape of the RNN.
    Note: The layer has been tested with Keras 2.0.6
    Example:
        model.add(LSTM(64, return_sequences=True))
        model.add(AttentionWithContext())
        # next add a Dense layer (for classification/regression) or whatever...
    """
 
    def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
 
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')
 
        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)
 
        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)
 
        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)
 
    def build(self, input_shape):
        assert len(input_shape) == 3
 
        self.W = self.add_weight((input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
 
        self.u = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)
 
        super(AttentionWithContext, self).build(input_shape)
 
    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None
 
    def call(self, x, mask=None):
        uit = dot_product(x, self.W)
 
        if self.bias:
            uit += self.b
 
        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)
 
        a = K.exp(ait)
 
        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())
 
        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())
 
        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)
 
    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]

In [0]:
from keras import backend as K
from keras.engine.topology import Layer

class Position_Embedding(Layer):
    
    def __init__(self, size=None, mode='sum', **kwargs):
        self.size = size #必须为偶数
        self.mode = mode
        super(Position_Embedding, self).__init__(**kwargs)
        
    def call(self, x):
        if (self.size == None) or (self.mode == 'sum'):
            self.size = int(x.shape[-1])
        batch_size,seq_len = K.shape(x)[0],K.shape(x)[1]
        position_j = 1. / K.pow(10000., \
                                 2 * K.arange(self.size / 2, dtype='float32' \
                               ) / self.size)
        position_j = K.expand_dims(position_j, 0)
        position_i = K.cumsum(K.ones_like(x[:,:,0]), 1)-1 #K.arange不支持变长，只好用这种方法生成
        position_i = K.expand_dims(position_i, 2)
        position_ij = K.dot(position_i, position_j)
        position_ij = K.concatenate([K.cos(position_ij), K.sin(position_ij)], 2)
        if self.mode == 'sum':
            return position_ij + x
        elif self.mode == 'concat':
            return K.concatenate([position_ij, x], 2)
        
    def compute_output_shape(self, input_shape):
        if self.mode == 'sum':
            return input_shape
        elif self.mode == 'concat':
            return (input_shape[0], input_shape[1], input_shape[2]+self.size)


class SelfAttention(Layer):

    def __init__(self, nb_head, size_per_head, **kwargs):
        self.nb_head = nb_head
        self.size_per_head = size_per_head
        self.output_dim = nb_head*size_per_head
        super(SelfAttention, self).__init__(**kwargs)

    def build(self, input_shape):
        self.WQ = self.add_weight(name='WQ', 
                                  shape=(input_shape[0][-1], self.output_dim),
                                  initializer='glorot_uniform',
                                  trainable=True)
        self.WK = self.add_weight(name='WK', 
                                  shape=(input_shape[1][-1], self.output_dim),
                                  initializer='glorot_uniform',
                                  trainable=True)
        self.WV = self.add_weight(name='WV', 
                                  shape=(input_shape[2][-1], self.output_dim),
                                  initializer='glorot_uniform',
                                  trainable=True)
        super(SelfAttention, self).build(input_shape)
        
    def Mask(self, inputs, seq_len, mode='mul'):
        if seq_len == None:
            return inputs
        else:
            mask = K.one_hot(seq_len[:,0], K.shape(inputs)[1])
            mask = 1 - K.cumsum(mask, 1)
            for _ in range(len(inputs.shape)-2):
                mask = K.expand_dims(mask, 2)
            if mode == 'mul':
                return inputs * mask
            if mode == 'add':
                return inputs - (1 - mask) * 1e12
                
    def call(self, x):
        #如果只传入Q_seq,K_seq,V_seq，那么就不做Mask
        #如果同时传入Q_seq,K_seq,V_seq,Q_len,V_len，那么对多余部分做Mask
        if len(x) == 3:
            Q_seq,K_seq,V_seq = x
            Q_len,V_len = None,None
        elif len(x) == 5:
            Q_seq,K_seq,V_seq,Q_len,V_len = x
        #对Q、K、V做线性变换
        Q_seq = K.dot(Q_seq, self.WQ)
        Q_seq = K.reshape(Q_seq, (-1, K.shape(Q_seq)[1], self.nb_head, self.size_per_head))
        Q_seq = K.permute_dimensions(Q_seq, (0,2,1,3))
        K_seq = K.dot(K_seq, self.WK)
        K_seq = K.reshape(K_seq, (-1, K.shape(K_seq)[1], self.nb_head, self.size_per_head))
        K_seq = K.permute_dimensions(K_seq, (0,2,1,3))
        V_seq = K.dot(V_seq, self.WV)
        V_seq = K.reshape(V_seq, (-1, K.shape(V_seq)[1], self.nb_head, self.size_per_head))
        V_seq = K.permute_dimensions(V_seq, (0,2,1,3))
        #计算内积，然后mask，然后softmax
        A = K.batch_dot(Q_seq, K_seq, axes=[3,3]) / self.size_per_head**0.5
        A = K.permute_dimensions(A, (0,3,2,1))
        A = self.Mask(A, V_len, 'add')
        A = K.permute_dimensions(A, (0,3,2,1))    
        A = K.softmax(A)
        #输出并mask
        O_seq = K.batch_dot(A, V_seq, axes=[3,2])
        O_seq = K.permute_dimensions(O_seq, (0,2,1,3))
        O_seq = K.reshape(O_seq, (-1, K.shape(O_seq)[1], self.output_dim))
        O_seq = self.Mask(O_seq, Q_len, 'mul')
        return O_seq
        
    def compute_output_shape(self, input_shape):
        return (input_shape[0][0], input_shape[0][1], self.output_dim)